# Implementation of a SQP for nonlinear optimal control
The goal of this exercise is to implement a SQP solver to solve a nonlinear optimal control problem.

Consider the pendulum below 

<img src='pendulum.png' width="150">

Assuming $m=l=1$, The dynamics of this pendulum is
$$\ddot{\theta} = u - g \sin\theta$$
which can be discretized with the following dynamics
$$\begin{align}\theta_{n+1} &= \theta_n + \Delta t \omega_n\\ 
\omega_{n+1} &= \omega_n + \Delta t (u_n - g \sin\theta_n)\end{align}$$
where $\theta_n$ is the angle of the pendulum with respect to the vertical at time step $n$ and $\omega_n$ its angular velocity. We will use $\Delta t = 0.01$.
The pendulum starts at configuration $\theta_0 = \omega_0 = 0$, i.e. all the way down with zero velocity and we would like to find
an optimal control that will bring it up to $\theta=\pi$ with zero velocities.

To get the pendulum to do this movement, we write the following optimal control problem
$$\begin{align}
& \min_{\theta_n, \omega_n, u_n} \sum_{n=0}^{300} 10(\theta_n - \pi)^2 + 0.1\omega_n^2 + 0.1u_n^2\\
\textrm{subject to}\ \ & \theta_{n+1} = \theta_n + \Delta t \ \omega_n \\
& \omega_{n+1} = \omega_n + \Delta t\ (u_n - g \sin\theta_n)\\
& \theta_0 = \omega_0 = 0
\end{align}$$

## Question 1: write a SQP solver to solve this problem
To do so, please follow these steps:
* Write down the algorithm (in words not in code), i.e. write all the steps you need to take
* Write (in Latex) the gradient of the running cost at a given guess $\bar{x} = [\bar{\theta}_0, \bar{\omega}_0, \bar{u}_0, \bar{\theta}_1, \bar{\omega}_1, \bar{u}_1, \dots, \bar{\theta}_{300}, \bar{\omega}_{300}, \bar{u}_{300}]^T$, i.e. for given values $\bar{\theta}_n, \bar{\omega}_n, \bar{u}_n$ and implement a function that computes it
* Write (in Latex) the Hessian of the running cost at a given guess $\bar{x}$, i.e. for given values $\bar{\theta}_n, \bar{\omega}_n, \bar{u}_n$ and implement a function that computes it
* Assume that the Hessian of the constraints is 0 (i.e. ignore the second order derivatives of the constraints)
* Write (in Latex) a linear approximation of the constraints at a given guess $\bar{x}$ in a form $G(\bar{x}) \Delta x = g(\bar{x})$ (don't forget the constant terms in g) where $\Delta x$ represents a small variation around $\bar{x}$ and implement a function that computes both $G$ and $g$.
* Use these functions to construct the inner linear KKT system that you will solve using Numpy's solve function (this should resemble the KKT system you built in the first homework)
* Implement a function that computes the amount of constraint violation, i.e. the sum of the absolute values of all the constraints (i.e. assuming constraints of the form $c(x) = 0$ we want to compute $|c(x)|$).
* Implement a filter linear search to test if a step should be accepted. You will implement the (simplified) filter line search explained below.
* Terminate the algorithm when you either reached the maximum number of iterations (e.g. 100) or when the KKT optimality conditions $\nabla_x L$ and $\nabla_\lambda L$ are close to 0, e.g. $10^{-4}$.


Once you have a solution, make sure to check that it satisfies the constraints! You can also use the function ``pendulum.animate_robot`` to display the pendulum motion. Please answer the following questions:
1. How many iterations did it take?
2. Plot the solution (angle, velocity and control)
3. Plot the amont of constraint violation per iteration of the solver
4. Plot the cost per iteration of the solver
5. Plot $\alpha$ for each iteration of the solver

### (Simple) filter linear search
Once you have a potential step $p_x$ and associated candidate Lagrange multipliers $p_\lambda$ (from the ``solve`` of the KKT system), you need to find a step $\alpha$ to update your guess of the solution $x_{guess}$ and the Lagrange multipliers $\lambda_{guess}$. We will accept a step that either reduces the amount of constraint violation or reduces the cost.

Let's denote $f(x)$ the cost at $x$ and $|c(x)|$ the amount of constraint violation at $x$. Initialize the variable $f_{best} = \infty$ and $c_{best}=\infty$ at the beginning of the SQP. 

Then do the following during the line search.
1. Set $\rho$ to a number between 0 and 1 (e.g. 0.5) and set $\alpha = 1$
2. If $f(x_{guess} + \alpha p_x) < f_{best}$ then set $f_{best} \leftarrow f(x_{guess} + \alpha p_x)$ and accept the step

   Or 

   If $|c(x_{guess} + \alpha p_x)| < c_{best}$ then set $c_{best} \leftarrow |c(x_{guess} + \alpha p_x)|$ and accept the step
3. If the step was not accepted set $\alpha \leftarrow \rho \alpha$ and go back to Step 2.
4. If the step was accepted update the guess $x_{guess} \leftarrow x_{guess} + \alpha p_x$ and the Lagrange multipliers $\lambda_{guess} \leftarrow (1-\alpha)\lambda_{guess} + \alpha p_{lambda}$

## Question 2: write a SQP solver with inequality constraints
Modify your SQP solver in order to enforce the additional constraint $-4 \leq u_n \leq 4$. 

In this case you will need to use a QP solver instead of the ``solve`` function. Please use the [qpsolvers](https://pypi.org/project/qpsolvers/) library (use ``pip install qpsolvers`` to get the latest version 4.4.0 and use ``cvxopt`` as QP solver). You may access the Lagrange multipliers of the solution following [this example](https://qpsolvers.github.io/qpsolvers/quadratic-programming.html#dual-multipliers).

Update the convergence checks accordingly (using the KKT condition for the nonlinear problem $\nabla_x L$). Also update the computation of the constraint violation by computing the amount of inequality constraint violation in absolute value (note that it should be zero when the constraint is satisfied).

Once you have a solution, make sure to check that it satisfies the constraints! You can also use the function ``pendulum.animate_robot`` to display the pendulum motion. Please answer the following questions:
1. How many iterations did it take?
2. Plot the solution (angle, velocity and control)
3. Plot the amont of constraint violation per iteration of the solver
4. Plot the cost per iteration of the solver
5. Plot $\alpha$ for each iteration of the solver
6. Compare this solution with the solution from Question 1. Are there any qualitative differences in the pendulum behavior? Did the solver converge faster or slower?

In [1]:
# Import necessary libraries
%matplotlib widget

import numpy as np
from qpsolvers import solve_qp, Problem, solve_problem
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.linalg import block_diag
import pendulum 


N = 300 
dt = 0.01
g = 9.81
tol = 1e-4

In [2]:
def state_function(_x):
    Q = np.array([[20, 0], [0, 0.2]])
    R = np.array([0.2])
    des_state = np.array([np.pi, 0, 0]).reshape(-1,1)
    mini_G = np.block([[Q, np.zeros((2,1))],[np.zeros((1,2)), R]])
    G = np.zeros((N*3,N*3))
    for i in np.arange(0, 3*N, 3):
        G[i:i+3, i:i+3] = mini_G 
    g = np.tile(-np.dot(mini_G, des_state), N).reshape(-1,1)
    state = ((0.5 * ((_x.T @ G) @ _x)) + (g.T @ _x))[0,0]
    return state

In [3]:
# Q = np.array([[20, 0], [0, 0.2]])
# R = np.array([0.2])
# des_state = np.array([np.pi, 0, 0]).reshape(-1,1)
# mini_G = np.block([[Q, np.zeros((2,1))],[np.zeros((1,2)), R]])
# # print(mini_G)

# g.shape

In [4]:
def grad_Cost_Matrix(_x):
    if not isinstance(_x, np.ndarray):
        print("State not passed properly")
    else:
        # Define the weighting matrix Q
        Q = np.array([[20, 0], [0, 0.2]])
        R = np.array([0.2])
        mini_G = np.block([[Q, np.zeros((2,1))],[np.zeros((1,2)), R]])
        
        # Define the desired state vector x_des
        x_des = np.array([np.pi, 0, 0]).reshape(-1,1)
        G = np.zeros((N*3,N*3))
        # g = np.zeros((N*3,1))
        for i in np.arange(0, 3*N, 3):
            G[i:i+3, i:i+3] = mini_G   
        # Compute the vector g by repeating (Q @ x_des) N times
        # (Q @ x_des) results in a (3, 1) vector
        # After flattening and tiling, g has shape (3*N, 1)
        # for i in np.arange(0,3*N, 3):
        #     g[i:i+3] = G @ x_des
        
        g = np.tile((-x_des.T @ mini_G).flatten(), N).reshape(-1, 1)

        # Compute and return the gradient of the cost function: ∇f(x) = G * _x + g
        # _x is expected to be a (3*N, 1) vector
        grad_f = (np.dot(G, _x) + g).reshape(-1,1)
    return grad_f

In [5]:
def GgSparse(_x: np.ndarray, N=300):
    if not isinstance(_x, np.ndarray):
        raise ValueError("State not passed properly")

    G_sparse = np.zeros((2,3*N))
    g_sparse = np.zeros((2,1))

    G_sparse[0, 0] = G_sparse[1, 1] = 1 

    # Loop over each time step to add dynamic constraints
    for i in range(0, (N - 1) * 3, 3):
        # Extract state and control variables from _x
        _theta_i = _x[i]        # theta_n
        _omega_i = _x[i + 1]    # omega_n
        _u_i = _x[i + 2]        # u_n
        _theta_new = _x[i + 3]  # theta_{n+1}
        _omega_new = _x[i + 4]  # omega_{n+1}

        
        mini_G = np.hstack((np.zeros((2, (i//3)*3)), np.array([[1, dt, 0, -1, 0, 0], [-(dt*g*np.cos(_theta_i)[0]), 1, dt, 0, -1, 0]]), np.zeros((2, (N-2-(i//3))*3))))
        # mini_G = np.hstack((np.zeros((2, (i//3)*3)), np.array([[1, dt, 0, -1, 0, 0], [-(dt*g*np.cos(bar_theta_i)[0]), 1, dt, 0, -1,0]]), np.zeros((2, (N - 2 - (i // 3)) * 3))))
        G_sparse = np.vstack((G_sparse, mini_G))
        mini_g = np.vstack(([_theta_i + (dt*_omega_i)-_theta_new], [_omega_i + (dt*(_u_i-(g*np.sin(_theta_i))))-_omega_new]))
        g_sparse = np.vstack((g_sparse, mini_g))
    print(G_sparse.shape)  # Should print (602, 900)
    print(g_sparse.shape)  # Should print (602, 1)

    return G_sparse, g_sparse

In [6]:
# # N = 4
# # Q = np.array([[20, 0], [0, 0.2]])
# # R = np.array([0.2])

# # G_sparse = np.zeros((2, 3 * N))
# # # Set the first element of G corresponding to theta_0 = 0
# # G_sparse[0, 0] = 1

# # # Set the second element of G corresponding to omega_0 = 0
# # G_sparse[1, 1] = 1
# # print(G_sparse)
# # del_t = 1
# # g = 9.81
# # del_t = 000.1
# _x = np.ones((5, 1))
# # # i=0
# # # Extract the current state and control from _x
# _theta_i = _x[0].squeeze()       # theta_n
# _omega_i = _x[1].squeeze()   # omega_n
# _u_i = _x[2].squeeze()       # u_n
# _theta_new = _x[3].squeeze()  # theta_{n+1}
# _omega_new = _x[4].squeeze()  # omega_{n+1}

# g_sparse = np.zeros((2,1))
# # # mini_g = np.array([[_theta_i + (dt*_omega_i)-_theta_new],[_omega_i + (dt*(_u_i-(g*np.sin(_theta_i))))- _omega_new]])
# # for i in range(N-1):
# #     mini_g = np.array([[_theta_i + (dt*_omega_i) - _theta_new], [_omega_i + (dt*(_u_i-g*np.sin(_theta_i))) - _omega_new]])
# #     g_sparse = np.vstack((g_sparse, mini_g))

# mini_g = np.array([[_theta_i + (dt*_omega_i)-_theta_new],[_omega_i + (dt*(_u_i-(g*np.sin(_theta_i))))- _omega_new]])
# g_sparse = np.vstack((g_sparse, mini_g))

# print(g_sparse)

In [7]:
def hess_lagrangian():
    mini_L = np.array([[20,0,0],[0, 0.2, 0],[0, 0 ,0.2]])
    L = np.zeros((3*N,3*N))
    for i in np.arange(0,3*N,3):
        L[i:i+3, i:i+3] = mini_L
    return L

# hess_lagrangian()


In [8]:
# def compute_hessian_L():
#     """Function to compute the hessian of the lagrangian, ignoring second order derivatives of the constraints

#     Returns:
#         np.ndarray: Hessian of the lagrangian
#     """
    
#     return 2 * block_diag(*([np.array([[20, 0, 0], [0, 0.2, 0], [0, 0, 0.2]])] * N))

In [9]:

def solve_KKT(_x):
    if not isinstance(_x, np.ndarray):
        print("State not passed properly")
    else:
        L = hess_lagrangian()
        G_sparse, g_sparse = GgSparse(_x)
        cost_grad_mat = grad_Cost_Matrix(_x)
        print("L",L.shape)
        print(G_sparse.shape)
        LHS = np.block([[L, G_sparse.T], [G_sparse, np.zeros((G_sparse.shape[0], G_sparse.shape[0]))]])
        RHS = np.vstack([-cost_grad_mat, -g_sparse])
        res = np.linalg.solve(LHS, RHS)
        pk = res[:3*N, 0]
        lambda_kp1 = res[3*N:, 0]
        Delta_lambda = res[L.shape[0]:]


# (600, 902)
# (600, 1)
# (900, 900)
# (600, 902)
# (600, 1)
# (900, 1)

    return pk, lambda_kp1

In [10]:
def tot_constraint_violation_eq(_x: np.ndarray):
    """
    Computes the total constraint violation for the current guess of optimization variables.

    This function calculates the sum of the absolute values of all constraint residuals
    evaluated at the current guess `_x`. It provides a measure of how much the current
    solution violates the constraints, which is useful for assessing feasibility and
    convergence in the SQP algorithm.

    Parameters
    ----------
    _x : np.ndarray
        Current guess of the optimization variables, structured as
        [θ₀, ω₀, u₀, θ₁, ω₁, u₁, ..., θₙ, ωₙ, uₙ]^T,
        where θ is the angle, ω is the angular velocity, and u is the control input at each time step.

    Returns
    -------
    float
        The total constraint violation, calculated as the absolute sum of all constraint residuals.
    """
    # Compute the Jacobian matrix of the constraints and the constraint residuals at `_x`
    _, g = GgSparse(_x)

    # Calculate the total constraint violation by summing the absolute values of the residuals
    total_violation = np.sum(abs(g))

    return total_violation

In [11]:
def cost_func(_x: np.ndarray, N=300):
    # Define the weighting matrix Q
    Q = np.array([
        [20, 0, 0],
        [0, 0.2, 0],
        [0, 0, 0.2]
    ])

    # Define the desired state vector x_des
    x_des = np.array([
        [np.pi],
        [0],
        [0]
    ])

    # Construct the block diagonal matrix G with N copies of Q along the diagonal
    # Shape of G: (3*N, 3*N)
    G = block_diag(*([Q] * N))

    # Compute the vector g by repeating (-x_des.T @ Q) N times
    # (-x_des.T @ Q) results in a (3, 1) vector
    # After flattening and tiling, g has shape (3*N, 1)
    g = np.tile((-x_des.T @ Q).flatten(), N).reshape(-1, 1)

    return ((0.5 * ((_x.T @ G) @ _x)) + (g.T @ _x))[0, 0]

In [12]:
# Initialize the guess for the decision variables (theta, omega, control inputs)
# Assuming each time step has 3 variables: theta (angle), omega (angular velocity), and u (control input)
# Therefore, for N1 time steps, the total number of variables is 3*N1
x_guess = np.zeros((3 * N, 1))  # Initial guess set to zero for all variables

# Initialize the guess for the Lagrange multipliers (dual variables)
# There are two equality constraints per time step: dynamics for theta and omega
# Additionally, there are 2*N1 inequality constraints for control input bounds (u_max and u_min)
# Initial guess set to zero for all dual variables
lambda_guess = np.zeros(((2 + (2 * N)), 1))

# Initialize line search parameters
alpha = 1       # Initial step size
rho = 0.5       # Reduction factor for step size during backtracking line search

# Initialize best cost and constraint violation to infinity
# These will be updated to keep track of the best (lowest) cost and lowest constraint violation achieved
c_best = np.inf  # Best (minimum) constraint violation found so far
cost_best = np.inf  # Best (minimum) cost found so far

# Initialize history lists to store cost, step size (alpha), and constraint violations across iterations
cost_history = []      # List to store the cost at each iteration
alpha_history = []     # List to store the step size alpha at each iteration
c_history = []         # List to store the constraint violation at each iteration

# Start the Sequential Quadratic Programming (SQP) optimization loop
for i in range(0, N):
    # Solve the Karush-Kuhn-Tucker (KKT) conditions to obtain the search direction (pk) and dual variables (pLambda)
    # This function should return the step direction and updated dual variables based on current guess
    pk, pLambda = solve_KKT(x_guess)

    # Reshape pk to ensure it has the correct dimensions (3*N1 x 1)
    # The number 900 implies that N1 = 300; however, it's better to use 3*N1 for flexibility
    pk = pk.reshape(900, 1)  # Reshaping the search direction vector

    # Perform backtracking line search to find an appropriate step size (alpha)
    # The line search aims to ensure that both the cost decreases and constraint violations are minimized
    while ((cost_func(x_guess + (alpha * pk)) > cost_best) &
           (tot_constraint_violation_eq(x_guess + (alpha * pk)) > c_best)):
        # Reduce the step size by multiplying with rho (0.5)
        alpha = rho * alpha

    # Update the best constraint violation and best cost based on the new candidate solution
    # Compute total constraint violation
    c_best = tot_constraint_violation_eq(x_guess + (alpha * pk))
    # Compute the cost of the new candidate solution
    cost_best = cost_func(x_guess + (alpha * pk))

    # Record the history of cost, alpha, and constraint violation for analysis
    alpha_history.append(alpha)          # Store the current step size
    # Store the current constraint violation
    c_history.append(c_best)
    cost_history.append(cost_best)       # Store the current cost

    # Print the current cost and constraint violation for monitoring progress
    print(
        f"Current Cost = {cost_best}  Current Constraint Violation = {c_best}")

    # Update the current guess of the decision variables by taking a step in the direction of pk scaled by alpha
    x_guess = x_guess + (alpha * pk)  # Update the solution vector

    # Check if the current constraint violation is below the specified tolerance (tol)
    if (c_best < tol):
        break  # If constraints are sufficiently satisfied, terminate the optimization loop

    # Update the guess for the dual variables (Lagrange multipliers) using a convex combination
    # This helps in stabilizing the dual variable updates
    lambda_guess = ((1 - alpha) * lambda_guess) + \
        (alpha * pLambda)  # Update dual variables

# After completing the iterations, print the total number of iterations performed
print(f"Total iterations needed = {i+1}")

# Extract the optimized variables for analysis and visualization
# Assuming the ordering of variables in x_guess is [theta_0, omega_0, u_0, theta_1, omega_1, u_1, ..., theta_N1-1, omega_N1-1, u_N1-1]
# Extract theta (angle) variables every 3 steps starting at index 0
theta = x_guess[0::3].T
# Extract omega (angular velocity) variables every 3 steps starting at index 1
omega = x_guess[1::3].T
# Extract control input (u) variables every 3 steps starting at index 2
controls = x_guess[2::3].T

# At this point, you can proceed to plot or analyze theta, omega, and control inputs as needed

(600, 900)
(600, 1)
L (900, 900)
(600, 900)
(600, 900)
(600, 1)
(600, 900)
(600, 1)
Current Cost = -14927.779426619058  Current Constraint Violation = 21.5740835377162
(600, 900)
(600, 1)
L (900, 900)
(600, 900)
(600, 900)
(600, 1)
(600, 900)
(600, 1)
Current Cost = -23828.810833427106  Current Constraint Violation = 17.887520840583125
(600, 900)
(600, 1)
L (900, 900)
(600, 900)
(600, 900)
(600, 1)
(600, 900)
(600, 1)
Current Cost = -24926.23248874456  Current Constraint Violation = 0.060137494823960545
(600, 900)
(600, 1)
L (900, 900)
(600, 900)
(600, 900)
(600, 1)
(600, 900)
(600, 1)
Current Cost = -24926.596929091236  Current Constraint Violation = 4.657671968136853e-06
Total iterations needed = 4


In [13]:
# Display the animation
x_init = np.array([theta[:, 0], omega[:, 0]])
pendulum.animate_robot(x_init, controls)

<source type="video/mp4" src="data:video/mp4;base64,AAAAIGZ0eXBNNFYgAAACAE00ViBpc29taXNvMmF2YzEAAAAIZnJlZQAAMQ9tZGF0AAACrwYF//+r
3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MyByMzA2MCA1ZGI2YWE2IC0gSC4yNjQvTVBF
Ry00IEFWQyBjb2RlYyAtIENvcHlsZWZ0IDIwMDMtMjAyMSAtIGh0dHA6Ly93d3cudmlkZW9sYW4u
b3JnL3gyNjQuaHRtbCAtIG9wdGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFs
eXNlPTB4MzoweDExMyBtZT1oZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVk
X3JlZj0xIG1lX3JhbmdlPTE2IGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBk
ZWFkem9uZT0yMSwxMSBmYXN0X3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTE5
IGxvb2thaGVhZF90aHJlYWRzPTMgc2xpY2VkX3RocmVhZHM9MCBucj0wIGRlY2ltYXRlPTEgaW50
ZXJsYWNlZD0wIGJsdXJheV9jb21wYXQ9MCBjb25zdHJhaW5lZF9pbnRyYT0wIGJmcmFtZXM9MyBi
X3B5cmFtaWQ9MiBiX2FkYXB0PTEgYl9iaWFzPTAgZGlyZWN0PTEgd2VpZ2h0Yj0xIG9wZW5fZ29w
PTAgd2VpZ2h0cD0yIGtleWludD0yNTAga2V5aW50X21pbj0xMCBzY2VuZWN1dD00MCBpbnRyYV9y
ZWZyZXNoPTAgcmNfbG9va2FoZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0w
LjYwIHFwbWluPTAgcXBtYXg9NjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAA
DNVliIQAP//+92ifAptaQ3qA5JXFJdtPgf+rZ3B8j+kDAAADAAADAAAVt6RlhkJ0L/JiAAAFfACy
BmSZ6XFJE4AKCdnVuDJz6YRA9aXK+D92cTMu+hvWEgrm6VgN+NN58xr5ENbD+oIFbJh3Hp5aaZ7q
A6T9bic4xbRxQpgkTkdJXn7mxrIyGimVgSkYPxFhDkBW2ULOVc3poxMubwa8KN+BHPPpeuCVceyT
K8Sutw9A6fdPtZ/8tSIN4rY0xxRToEK/HoC502qtOxtzT5Rshcw28vw//QkgQVxA2eXnAUAxIAOb
GLPoKfXjk5pFs/rBKwvOmK74JvarEOzfC34c3bf3d3EXiENvbfmgN445qoqXG0rX23WzVwbHgmtm
Q3VcS412auklSczhDcDlDnRxPyboQULE4aoDSPdIVIwcHKRFurb7GI3GohEQb37fenqBMoKtvGDc
ZQX/K0/AIDssxoDtSaQ9lJ/bFa/O6nXXKcpTKr+GKYKpYMHoFtBt5JcYBX6JL7HvFE23bTVFiJ2i
L2/SIrRv7xzo1biqog9FulZ9vPln70zwy1hTxAooHAEnuGRm+741AFjGQ4ELGQ/NJWF2pLTK+Ad1
vD8Pbev5HVEAb4r90LB+BOzJj0e61/EbH1E8xefu3/RleFlNwAww0NXSCQNCmpgBD9/cGehT78cf
fdZeSYfh4j42j3K8r7hTcJ8XPTimB/5TR2ewjoZ69kvbn0UVGO8BXBm5gBaaLUYjKX3AFjv/WSAB
1l9wgVwA4xaXuWAAAAMAAA0+Mx3VFhjCJ35zx2KPVXy/X//MBYzpt/ACmFNpYWvW4jC2HqbrV04K
tyiKfsqxHioMxH4PoOwM6Qc0VhicD8vNu8KxWQfc4eMl6XZQHH0Je/NllYEvtdtiOEp6TMoeHka/
dceDqNaR0raHZ7ptRX5klMO2E+/CdWCjEVZYZOL13BjJPtiXXYVAMRjEQDJbQywgRhPDZBP7SLEl
ygxcyZj6w2GBrxT1XThJPF3R50QInTtoXQA87gfHskEbov+Ar48TFGGmNYmMWsntuuZr/DowAGtR
3h6xwmo90/pEIR7DRszTyPSabgp5qZaFzi0xyqaBfXian/xuhhmOFA1YjDB8Fce0aUBwz5khlvYd
ggnMwuwC8SpP+imOj4+E6W2gda/p1Cy2nd0QMWdCtKaxi7auvDxPjoYWkc4pKAjW4bONBytSMqAA
AAMAAAMABdWM7EIQbX46gwWRhplLPIs7QIzJ3v7LbkMO7zmoZu3zqrsmGY0+TJXjy5CiQdFq4J4C
s1+X3j5nm+QAqoO7xT85N0yBXH7EJ+awnyC+OG6IAVmlzYAJUA3Zl2StFtGHCk1XJ+ly3WiYkC9Y
V5XjoWdP8qbdS2jGk2D+HmiXs+e5Au4x8aF2So32xIk/j9uIm2ZNEuXWd16RGX7bAfjEEWTXqGYB
bT4qp0EqkBs/WElZ55/Q/no7MUZrMcZTD+F82q2mBRgH0g5gidWOq8Iq0ceDE78pzB9dLaX5NCox
9cgH3ik+onHGMaYs8fLsY4Pc6DBfi4XNvnedUhU1T15Kjzw5LFUxh1S2L8PrtZDBPvH2vj9RK+UQ
LBmwF5ezChCEptTTXlvz7t7WQym7qwwNByZtAKJ6XJ8A2MQkeKzj+Bpu1YjrlrBVhHUM6u9ZLe8D
ZVFvsvP2N9t+Rg8ydyuSpkTLCS7TyB7xchC+/OJQxLXDOuSdLZ7I7WOOP1bb01BFPdMU337GTZng
Iu0vN5ZsBGxVj4KHzDen3d/vHHc8+CCao5w+lH7GGB//2DswifAEpUMeKUOWu8E6hr451ZwdpNLA
SzF4ajIqcLsSju5W4fBmeZGxR0OEJqh2Yqkv6YyCG5KuRPJAr+Cea2zcnLwybpdewyFmjiHWYz5b
bg9Hinjf5v7gblkp8QH9kWjwNxYZCQ6wG963CfMfscfV26tpO6oOEBVldqWCpSEeFD6RB25TyGPs
K/T3uvprsb6lKxv26Q8YBcGT3r/kdYmmtLRzIrefx9HhIsLy/dGe2Yk9lI3G0ZZ4012Jovutff1g
0QAR7A9qH5lhcOPTnU8TJzyUjhefAjHfdMQFqrtWNPMIwigmkd9sNHqGKVV7sI9zwkalCaKh31ZP
xk0HsnL1ptiXEt/VAWdthRsdFcwxhW3V7vUrMRetpxQ+QveoIbfmCRan5kwN+uNjwHP5/yHU5oYw
4NkgsG6YTpvJaR7jtsKE/vdi/e2NhW+0BXEXwqlHmpMhhTfpQQQ/g1acEkqlYwOmItjR0xavqw/l
4epMas47Q3gSFvIschyszGyxIX/cOa8u95KbIh1Mv5lxs+fADGo3IhH0llRSyOdfWL07bQEGKy+N
wuqsweq+5HZTKfDTbB+xtWQsXzBRdlBXv5vAAAOzfGsYjALPh5IyM8ug5vYLu/AAQOzKSG6YAAJd
DyLATAf1XwYZKO8+iYtZNu10R5XkwRIyPDow58Fl6COxzljLp8ZBuU39YmwQCZ5n1OZ5iYxKuJM9
g/OgnLfsf5enYu9XVCsMSJqhNDoof+MXwAg5lkoItDL4BD6pME4/X7HVeU3Qi0CSpsoi0+4Vxlur
4cf2NA6hN4uylDIUr69FvYatygLGre2n4bTrR4+hNrc/iXimQWZLFGJux+wbl9zLaKZTzzERrq/W
kV8xn+IAq4KjqC47kUybC2HNHNTChPkIFsywjq6EKSvKW5QtZ7tDnfnr95icaaE4+qVKkOjh7qXb
jU5bovU2WQXtimyZVPam3Zz9dLd+5mg4X6BaQFtyZPPDTn/4gPhqla3c8v//7u3NUS4dUFpovpZn
A07arf45p7U04c0kLh78n5MNw+MN/lGtR88un12aw9LZEnoYW71+QhPZHkRNMDPVcfuA9ocrszO6
i/2RiKNLTYtWNMtp8flk6

In [14]:
# dt is defined here
print(f'we use the following dt={pendulum.dt}')

# and g here
print(f'we use the following g={pendulum.g}')

# you can use this animate function to display what the pendulum would do for a given sequence of control
N = 300
controls = np.zeros((N,1))
x_init = np.array([[1.0],[0.]])
pendulum.animate_robot(x_init, controls.T)

we use the following dt=0.01
we use the following g=9.81


Your browser does not support the video tag.